In [20]:
import numpy as np
import pandas as pd
import h5py
import cv2

### Load SLEAP and Bonsai Data

In [33]:
mouse_id = "7004"
session_id = "m4"
experiment = "clickbait-motivate"
root_path = "S:/"

event_path = f"{root_path}/{experiment}/bonsai/{mouse_id}/{session_id}/events.csv"
sleap_path = f"{root_path}/{experiment}/sleap/{mouse_id}/{session_id}/7004_m4_sleap.000_062425_7004_m4.analysis.csv"

event_data = pd.read_csv(event_path)
sleap_data = pd.read_csv(sleap_path)

# Reindex to impute frames where SLEAP didn't find a a mouse instance
min_idx = sleap_data['frame_idx'].min()
max_idx = sleap_data['frame_idx'].max()
complete_range = range(min_idx, max_idx + 1)
# Set index column as the actual pandas index temporarily
df_reindexed = sleap_data.set_index('frame_idx').reindex(complete_range)
# Reset index to make it a regular column again
df_complete = df_reindexed.reset_index()
df_complete.rename(columns={'index': 'frame_idx'}, inplace=True)

sleap_data = df_complete.interpolate()  # Impute missing values with linear interpolation


In [34]:
event_data.head()

,trial_number,timestamp,poke_left,poke_right,centroid_x,centroid_y,target_cell,iti,water_left,water_right,reward_state,click,flip_state,speed,direction,drinking,timestamp_ms
0,0,2025-06-25 15:14:23.264371200,False,False,672,303,16.0,False,True,False,False,False,False,0.000000,0.000000,False,8259
1,0,2025-06-25 15:14:23.298073600,False,False,679,291,16.0,False,True,False,False,False,False,13.892444,1.570689,False,8292
2,0,2025-06-25 15:14:23.327820800,False,False,682,279,16.0,False,True,False,False,False,False,12.369317,1.570751,False,8325
3,0,2025-06-25 15:14:23.348633600,False,False,675,275,16.0,False,True,False,False,False,False,8.062258,0.785421,False,8359
4,0,2025-06-25 15:14:23.370880000,False,False,672,275,16.0,False,True,False,False,False,False,3.000000,0.000000,False,8392


In [35]:
sleap_data[-5:]

,frame_idx,track,instance.score,nose.x,nose.y,nose.score,centroid.x,centroid.y,centroid.score,tailbase.x,tailbase.y,tailbase.score
78918,78918,NaN,1.588075,357.057348,1744.105432,0.000000,504.624420,1684.380005,0.888498,596.663086,1596.383545,0.621944
78919,78919,NaN,1.598764,355.957920,1743.258335,0.000000,504.500336,1684.322998,0.885732,596.537231,1596.363525,0.630794
78920,78920,NaN,1.634210,354.858493,1742.411237,0.000000,504.331146,1684.206909,0.876170,596.137268,1592.485107,0.667485
78921,78921,NaN,1.601967,353.759065,1741.564139,0.000000,500.595245,1688.119995,0.871896,593.237183,1592.397339,0.642041
78922,78922,NaN,1.795430,352.659637,1740.717041,0.330317,504.169464,1688.426025,0.883555,596.473022,1592.727173,0.581558


### Load Video

In [48]:
video_path = f"{root_path}/{experiment}/bonsai/{mouse_id}/{session_id}/062425_7004_m4.avi"
cap = cv2.VideoCapture(video_path)

#  Video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video properties: {width}x{height}, {fps} FPS, {total_frames} frames")

Video properties: 888x1968, 30.0 FPS, 78923 frames


### Visualize SLEAP Tracking

In [50]:
print(total_frames)  # Total frames in video file
print(len(sleap_data))  # Rows in SLEAP data
print(len(event_data))  # Rows in Bonsai event data
offset = abs(len(event_data) - len(sleap_data))

78923
78923
78909


In [52]:
scale = 0.5  # Scaling factor for preview animation
nose = sleap_data[['nose.x', 'nose.y']].to_numpy().astype(np.int16)
cent = sleap_data[['centroid.x', 'centroid.y']].to_numpy().astype(np.int16)
base = sleap_data[['tailbase.x', 'tailbase.y']].to_numpy().astype(np.int16)
bon_cent = event_data[['centroid_x', 'centroid_y']].to_numpy().astype(np.int16)

# Toggle preview
save = False
# Toggle video background
video = True

for ii in range(offset, len(sleap_data)):
    if video:
        cap.set(cv2.CAP_PROP_POS_FRAMES, ii)  # Get current video frame
        ret, canvas = cap.read()
    else:
        canvas = np.zeros((1968,888,3), dtype=np.uint8)  # Clear canvas
    
    # Bonsai centroid
    cv2.circle(img=canvas, center=bon_cent[ii-offset], radius=12, color=(128,128,128), thickness=-1, lineType=cv2.LINE_AA)  # Centroid from realtime tracking
    # SLEAP
    cv2.circle(img=canvas, center=nose[ii], radius=5, color=(255,0,0), thickness=-1, lineType=cv2.LINE_AA)  # Nose
    cv2.circle(img=canvas, center=cent[ii], radius=5, color=(0,255,0), thickness=-1, lineType=cv2.LINE_AA)  # Centroid
    cv2.circle(img=canvas, center=base[ii], radius=5, color=(0,0,255), thickness=-1, lineType=cv2.LINE_AA)  # Tail base

    cv2.resize(canvas, dsize=(int(canvas.shape[1]*scale), int(canvas.shape[0]*scale)))
    canvas = cv2.rotate(canvas, cv2.ROTATE_90_CLOCKWISE) 

    if save:
        cv2.imwrite(f"S:/track-test/{mouse_id}_{session_id}_tracking_{ii}.png", canvas)
    else:
        cv2.imshow('SLEAP Test', canvas)
        # Wait for key press with timeout
        key = cv2.waitKey(16) & 0xFF
        # Break on 'q' key or ESC key
        if key == ord('q') or key == 27 or ii == len(sleap_data):
            break

cv2.waitKey(0)
cv2.destroyAllWindows()